In [1]:
import findspark
findspark.init('/home/mysparkub/spark-3.0.0-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler

In [3]:
spark = SparkSession.builder.appName('dogfood').getOrCreate()

In [4]:
data = spark.read.csv('files/dog_food.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [6]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [7]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [9]:
output = assembler.transform(data)

In [14]:
final_data = output.select('features', 'Spoiled')
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
rfc_model = RandomForestClassifier(numTrees=100, labelCol='Spoiled').fit(train_data)

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [19]:
preds = rfc_model.transform(test_data)

In [28]:
acc_eval = MulticlassClassificationEvaluator(labelCol='Spoiled',metricName='accuracy')

In [29]:
acc_eval.evaluate(preds)

0.9872611464968153

In [30]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0293, 1: 0.0222, 2: 0.9222, 3: 0.0263})